Required inputs for Akita are:
* binned Hi-C or Micro-C data stored in cooler format (https://github.com/mirnylab/cooler)
* Genome FASTA file

First, make sure you have a FASTA file available consistent with genome used for the coolers. Either add a symlink for a the data directory or download the machine learning friendly simplified version in the next cell.

In [2]:
import json
import os
os.environ["PATH"] = "/usr/local/cuda-11.5/bin:" + os.environ["PATH"]
os.environ["LD_LIBRARY_PATH"] = "/usr/local/cuda-11.5/lib64:" + os.environ.get("LD_LIBRARY_PATH", "")
import shutil
import subprocess
from importlib import reload
import sys

sys.path.append('/home/017448899/basenji/bin')  # Replace with the actual path
sys.path.append('/home/017448899/basenji/basenji')
import dataset
import akita_data
import basenji_data_write
import akita_train
reload(dataset)
reload(akita_data)
reload(basenji_data_write)
reload(akita_train)



2025-03-11 19:46:45.034937: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-03-11 19:46:46.892218: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/017448899/miniconda3/envs/basenji/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using TensorFlow device: /CPU:0
Using TensorFlow device: /CPU:0


<module 'akita_train' from '/home/017448899/basenji/bin/akita_train.py'>

In [3]:
import tensorflow as tf
import os
print(tf.__version__)
print("GPUs Available:", tf.config.list_physical_devices('GPU'))

2.13.1
GPUs Available: []


2025-03-11 19:46:59.344040: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


In [4]:
import sys
print(sys.executable)
import tensorflow as tf
print(tf.__version__)

/home/017448899/miniconda3/envs/basenji/bin/python
2.13.1


In [68]:
import pysam

# read FASTA
fasta_file = "./data/hg38.ml.fa"
fasta_open = pysam.Fastafile(fasta_file)
seqs_bed_file = './data/1m/sequences.bed'
model_seqs = []
for line in open(seqs_bed_file):
    a = line.split()
    model_seqs.append(ModelSeq(a[0],int(a[1]),int(a[2]),None))

    if options.end_i is None:
        options.end_i = len(model_seqs)

num_seqs = options.end_i - options.start_i
seq_dna = fasta_open.fetch(mseq.chr, mseq.start, mseq.end)
seq_dna = fetch_dna(fasta_open, mseq.chr, mseq_start, mseq_end)
print("This is the sequence's length {}".format(len(seq_dna)))

NameError: name 'ModelSeq' is not defined

In [104]:
from transformers import AutoTokenizer, AutoModel
import torch

# Initialize the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("zhihan1996/DNA_bert_6")
model = AutoModel.from_pretrained("zhihan1996/DNA_bert_6")

# Preprocess the DNA sequence into 6-mers
def split_into_kmers(sequence, k=6):
    return [sequence[i:i+k] for i in range(len(sequence) - k + 1)]

dna = "ACGTAGGTCAGTTCGATC"
kmers = split_into_kmers(dna, k=6)
print("K-mers:", kmers)

# Tokenize the k-mers
inputs = tokenizer(kmers, return_tensors="pt", padding=True, truncation=True)
#print("Tokenized inputs:", inputs)

# Pass through the model
outputs = model(**inputs)
print(outputs)
hidden_states = outputs.last_hidden_state  # Shape: [batch_size, sequence_length, hidden_size]
print("Hidden states shape:", hidden_states.shape)

# Mean pooling
embedding_mean = torch.mean(hidden_states, dim=1)  # Pool across sequence length
print("Mean pooling embedding shape:", embedding_mean.shape)  # Expect: [batch_size, hidden_size]

# Max pooling
embedding_max = torch.max(hidden_states, dim=1)[0]  # Pool across sequence length
print("Max pooling embedding shape:", embedding_max.shape)  # Expect: [batch_size, hidden_size]

K-mers: ['ACGTAG', 'CGTAGG', 'GTAGGT', 'TAGGTC', 'AGGTCA', 'GGTCAG', 'GTCAGT', 'TCAGTT', 'CAGTTC', 'AGTTCG', 'GTTCGA', 'TTCGAT', 'TCGATC']
BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[-0.8302, -0.0339,  0.8400,  ..., -1.5183,  0.1905, -1.6725],
         [-1.0396,  0.3682,  1.2651,  ..., -1.2533, -2.3157, -1.8425],
         [-0.1801,  0.1782,  0.4828,  ..., -1.0035, -2.0247, -1.7128]],

        [[-0.8531,  0.0451,  1.6959,  ..., -0.7248,  0.0168, -2.2951],
         [ 0.6211, -0.8345,  0.4039,  ..., -1.1610, -1.3098, -1.3100],
         [ 0.9651, -0.8113,  0.3069,  ..., -0.8097, -0.5686, -1.6427]],

        [[ 0.5663, -0.8957,  0.3128,  ...,  0.6162, -0.8174, -2.1774],
         [ 0.7426, -0.6788,  0.0847,  ...,  0.3084, -0.9915, -2.0305],
         [ 0.3851, -0.6906,  0.2997,  ...,  0.2675, -0.6946, -1.8679]],

        ...,

        [[-0.6784,  2.6782,  1.0214,  ..., -0.2528, -0.1694, -0.3810],
         [-0.3693,  1.5446,  0.6190,  ...,  0.0420, -0.4617,  0.4870

In [3]:
if not os.path.isfile('./data/hg38.ml.fa'):
    print('downloading hg38.ml.fa')
    subprocess.call('curl -o ./data/hg38.ml.fa.gz https://storage.googleapis.com/basenji_barnyard2/hg38.ml.fa.gz', shell=True)
    subprocess.call('gunzip ./data/hg38.ml.fa.gz', shell=True)

Download a few Micro-C datasets, processed using distiller (https://github.com/mirnylab/distiller-nf), binned to 2048bp, and iteratively corrected. 

In [4]:
if not os.path.exists('./data/coolers'):
    os.mkdir('./data/coolers')
if not os.path.isfile('./data/coolers/HFF_hg38_4DNFIP5EUOFX.mapq_30.2048.cool'):
    subprocess.call('curl -o ./data/coolers/HFF_hg38_4DNFIP5EUOFX.mapq_30.2048.cool'+
            ' https://storage.googleapis.com/basenji_hic/tutorials/coolers/HFF_hg38_4DNFIP5EUOFX.mapq_30.2048.cool', shell=True)
    subprocess.call('curl -o ./data/coolers/H1hESC_hg38_4DNFI1O6IL1Q.mapq_30.2048.cool'+
            ' https://storage.googleapis.com/basenji_hic/tutorials/coolers/H1hESC_hg38_4DNFI1O6IL1Q.mapq_30.2048.cool', shell=True)

In [11]:
ls ./data/coolers/

H1hESC_hg38_4DNFI1O6IL1Q.mapq_30.2048.cool
HFF_hg38_4DNFIP5EUOFX.mapq_30.2048.cool


Write out these cooler files and labels to a samples table.

In [6]:
lines = [['index','identifier','file','clip','sum_stat','description']]
lines.append(['0', 'HFF', './data/coolers/HFF_hg38_4DNFIP5EUOFX.mapq_30.2048.cool', '2', 'sum', 'HFF'])
lines.append(['1', 'H1hESC', './data/coolers/H1hESC_hg38_4DNFI1O6IL1Q.mapq_30.2048.cool', '2', 'sum', 'H1hESC'])

samples_out = open('data/microc_cools.txt', 'w')
for line in lines:
    print('\t'.join(line), file=samples_out)
samples_out.close()

Next, we want to choose genomic sequences to form batches for stochastic gradient descent, divide them into training/validation/test sets, and construct TFRecords to provide to downstream programs.

The script [akita_data.py](https://github.com/calico/basenji/blob/master/bin/akita_data.py) implements this procedure.

The most relevant options here are:

| Option/Argument | Value | Note |
|:---|:---|:---|
| --sample | 0.1 | Down-sample the genome to 10% to speed things up here. |
| -g | data/hg38_gaps_binsize2048_numconseq10.bed | Dodge large-scale unmappable regions determined from filtered cooler bins. |
| -l | 1048576 | Sequence length. |
| --crop | 65536 | Crop edges of matrix so loss is only computed over the central region. |
| --local | True | Run locally, as opposed to on a SLURM scheduler. |
| -o | data/1m | Output directory |
| -p | 8 | Uses multiple concourrent processes to read/write. |
| -t | .1 | Hold out 10% sequences for testing. |
| -v | .1 | Hold out 10% sequences for validation. |
| -w | 2048 | Pool the nucleotide-resolution values to 2048 bp bins. |
| fasta_file| data/hg38.ml.fa | FASTA file to extract sequences from. |
| targets_file | data/microc_cools.txt | Target table with cooler paths. |

Note: make sure to export BASENJIDIR as outlined in the basenji installation tips 
(https://github.com/calico/basenji/tree/master/#installation). 

In [5]:
if os.path.isdir('data/1m'):
    shutil.rmtree('data/1m')

In [ ]:
if os.path.isdir('data/1m_onehot'):
    shutil.rmtree('data/1m_onehot')

OSError: [Errno 16] Device or resource busy: '.nfs0000000012125faf0001ef5b'

In [6]:
! /home/017448899/basenji/bin/akita_data.py --sample 0.75 -g ./data/hg38_gaps_binsize2048_numconseq10.bed -l 262144 --crop 16384 --local -o ./data/1m --as_obsexp -p 8 -t .1 -v .1 -w 2048 --snap 2048 --stride_train 262144 --stride_test 524288 ./data/hg38.ml.fa ./data/microc_cools.txt

Contigs divided into
 Train:   526 contigs, 2136321418 nt (0.8009)
 Valid:    56 contigs,  265659136 nt (0.0996)
 Test:     61 contigs,  265472768 nt (0.0995)
writing sequences to BED
/home/017448899/basenji/bin/akita_data_read.py --crop 16384 -d 2 -k 0 -w 2048 --clip 2.000000 --as_obsexp ./data/coolers/HFF_hg38_4DNFIP5EUOFX.mapq_30.2048.cool ./data/1m/sequences.bed ./data/1m/seqs_cov/0.h5
/home/017448899/basenji/bin/akita_data_read.py --crop 16384 -d 2 -k 0 -w 2048 --clip 2.000000 --as_obsexp ./data/coolers/H1hESC_hg38_4DNFI1O6IL1Q.mapq_30.2048.cool ./data/1m/sequences.bed ./data/1m/seqs_cov/1.h5
/home/017448899/miniconda3/envs/basenji/lib/python3.8/site-packages/cooltools/lib/numutils.py:1376: RuntimeWarning: invalid value encountered in divide
  val_cur = ar_cur / armask_cur
/home/017448899/miniconda3/envs/basenji/lib/python3.8/site-packages/cooltools/lib/numutils.py:1376: RuntimeWarning: invalid value encountered in divide
  val_cur = ar_cur / armask_cur
These are the fold labels [

The data for training is now saved in data/1m as tfrecords (for training, validation, and testing), where *contigs.bed* contains the original large contiguous regions from which training sequences were taken, and *sequences.bed* contains the train/valid/test sequences.

In [1]:
! cut -f4 data/1m/sequences.bed | sort | uniq -c

      5 test
     83 train
      5 valid


In [2]:
! cut -f4 data/1m/sequences.bed | sort | uniq -c

      5 test
     83 train
      5 valid


In [3]:
! head -n3 data/1m/sequences.bed

chr8	97886208	97947648	train
chr8	22872064	22933504	train
chr1	180869120	180930560	train


In [4]:
! head -n3 data/1m/sequences.bed

chr8	97886208	97947648	train
chr8	22872064	22933504	train
chr1	180869120	180930560	train


Now train a model!

(Note: for training production-level models, please remove the --sample option when generating tfrecords)

In [18]:
# specify model parameters json to have only two targets
import json
params_file   = './params.json'
with open(params_file) as params_file:
    params_tutorial = json.load(params_file)   
params_tutorial['model']['head_hic'][-1]['units'] =2
with open('./data/1m/params_tutorial.json','w') as params_tutorial_file:
    json.dump(params_tutorial,params_tutorial_file) 
    
### note that training with default parameters requires GPU with >12Gb RAM ###

In [16]:
# specify model parameters json to have only two targets
import json
params_file   = './params.json'
with open(params_file) as params_file:
    params_tutorial = json.load(params_file)   
params_tutorial['model']['head_hic'][-1]['units'] =2
with open('./data/1m_onehot/params_tutorial.json','w') as params_tutorial_file:
    json.dump(params_tutorial,params_tutorial_file) 
    
### note that training with default parameters requires GPU with >12Gb RAM ###

In [13]:
! /home/017448899/basenji/bin/akita_train.py -k -o ./data/1m/train_out/  ./data/1m/params_tutorial.json ./data/1m

2025-03-08 02:08:10.059678: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-03-08 02:08:11.704724: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
./data/1m/params_tutorial.json
HI

Dataset Parameters:
seq_depth: 4
seq_1hot: True
target_length: 210
num_targets: 2
pool_width: 2048

HELLO
2025-03-08 02:08:14.023931: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
🔹 Original sequence shape: (None,)
✅ Fixed sequence shape: (62, 768)
Loading dataset from: ./data/1m
Features shape: (2, 62, 768)
Labels shape: (2, 210, 2)
x shape: (2, 62, 768)
y shape: (2, 210, 2)
HI

Dataset Parameters:
seq_depth: 4
seq_1hot: Tr

In [1]:
! /home/017448899/basenji/bin/akita_train.py -k -o ./data/1m_onehot/train_out/  ./data/1m_onehot/params_tutorial.json ./data/1m_onehot

2025-03-10 18:34:20.771359: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-03-10 18:34:22.511286: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Traceback (most recent call last):
  File "/home/017448899/basenji/bin/akita_train.py", line 27, in <module>
    import tensorflow as tf
  File "/home/017448899/miniconda3/envs/basenji/lib/python3.8/site-packages/tensorflow/__init__.py", line 38, in <module>
    from tensorflow.python.tools import module_util as _module_util
  File "/home/017448899/miniconda3/envs/basenji/lib/python3.8/site-packages/tensorflow/python/__init__.py", line 45, in <module>
    from tensorflow.python.feature_column import feature_column_lib as feature_column
  File "/home/01744889

See explore_model.ipynb for tips on investigating the output of a trained model. 